In [20]:
import pandas as pd
import numpy as np

#### Read Data

In [21]:
df = pd.read_csv('D:\FirstSemMT\ML\Assignments\DLBCL.tab',sep='\t')
df.shape

(79, 7071)

In [22]:
df.head()

,A28102,AB000114_at,AB000115_at,AB000220_at,AB000409_at,AB000449_at,AB000450_at,AB000460_at,AB000462_at,AB000464_at,...,U58516_at,U73738_at,X06956_at,X16699_at,X83863_at,Z17240_at,L49218_f_at,M71243_f_at,Z78285_f_at,class
0,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,...,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,DLBCL FL
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,class
2,-1,-45,176,97,-57,233,265,945,56,819,...,1036,-161,1523,-72,897,366,3,76,-28,DLBCL
3,25,-17,531,353,122,155,209,1688,42,639,...,4254,-75,6056,-68,737,81,-4,41,-45,DLBCL
4,73,91,257,80,614,507,760,2252,196,863,...,1934,-129,2901,-9,946,450,38,164,-62,DLBCL


In [23]:
df.drop(df.index[df['class'] == 'DLBCL FL'], inplace = True)
df.drop(df.index[df['class'] == 'class'], inplace = True)
label = df['class'] 
df.drop(['class'], axis = 1, inplace = True)
df = df.values
samples,features = df.shape
print(samples,features)

77 7070


In [24]:
data = df.astype(np.float)
data.shape

(77, 7070)

#### Import train_test_split function and Split dataset into training set and test set

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2,random_state=10) 

#### Find T score for every feature 

In [26]:
c1 = np.where(y_train=='DLBCL')[0]
c2 = np.where(y_train=='FL')[0]
m1 = X_train[c1,:].mean(axis=0)
m2 = X_train[c2,:].mean(axis=0)
v1 = X_train[c1,:].var(axis=0)
v2 = X_train[c2,:].var(axis=0)
tscore = np.abs(m1-m2)/np.sqrt(v1/c1.shape[0] + v2/c2.shape[0])
tscore

array([1.02319343, 0.66813182, 0.84204914, ..., 0.1229621 , 0.93524061,
       0.25709049])

#### Sort indexes in descending order of T score

In [27]:
sort_idxs=(-tscore).argsort()[:tscore.shape[0]]
sort_idxs

array([3968, 6119, 4232, ..., 4935, 1775, 6997], dtype=int64)

#### Select top features (less than 20%)

In [28]:
filt_idxs = sort_idxs[0:int((data.shape[1])*0.2)]
filt_idxs

array([3968, 6119, 4232, ..., 2607, 1236, 1105], dtype=int64)

In [29]:
X_new_train = X_train[:,filt_idxs]
X_new_test = X_test[:,filt_idxs]
X_new_train.shape

(61, 1414)

#### KNN Classification

In [30]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_new_train, y_train)
y_pred = knn.predict(X_new_test)

#### Accuracy, F score and Confusion matrix using KNN

In [31]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("f1 score:",metrics.f1_score(y_test, y_pred, average="macro"))
print("confusion matrix:",metrics.confusion_matrix(y_test, y_pred))

Accuracy: 0.8125
f1 score: 0.7257142857142858
confusion matrix: [[11  3]
 [ 0  2]]


#### SNM Classification

In [32]:
from sklearn import svm
clf = svm.SVC(kernel='linear')
clf.fit(X_new_train, y_train)
y_pred = clf.predict(X_new_test)

#### Accuracy, F score and Confusion matrix using SVM

In [33]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("f1 score:",metrics.f1_score(y_test, y_pred, average="macro"))
print("confusion matrix:",metrics.confusion_matrix(y_test, y_pred))

Accuracy: 1.0
f1 score: 1.0
confusion matrix: [[14  0]
 [ 0  2]]
